### Task 105: Simulate Non-rebasing token being deposited and traded against rebasing token
* Simulate CPT trading with a non-rebasing toke paired with a rebasing token.

In [1]:
import os
import copy
import numpy as np
import time
import matplotlib.pyplot as plt
cwd =  os.getcwd().replace("notebooks/test","")
os.chdir(cwd)

from python.dev.event.state.series import StateSeries
from python.dev.event.state import State
from python.dev.event import Deposit
from python.dev.event import Withdraw
from python.dev.token.trade import Convert
from python.dev.cpt import Liquidity
from python.dev.cpt import PriceCurve
from python.dev.cpt import ConstantProductTrade
from python.dev.plot import PlotPriceCurve
from python.dev.token import RebaseToken
from python.dev.token import NonRebaseToken
from python.dev.math.model import TimeDeltaModel
from python.dev.math.model import TokenDeltaModel
from python.dev.math.model import EventSelectionModel

### Part A: individually step through each of the instructions

* Initialize Rebasing and Non-rebasing tokens along with liquidity pool

In [2]:
# Initialize Rebasing/NonRebasing Tokens
APY = 0.1

TYPE_DEPOSIT_TOKEN = 'DEPOSIT'
TYPE_COLLATERAL_TOKEN = 'COLLATERAL'

time_delta = TimeDeltaModel().apply()
depositT1 = 10000  

T1 = NonRebaseToken(TYPE_DEPOSIT_TOKEN)
t1Address = T1.mint(depositT1)

time_delta = TimeDeltaModel().apply()
depositTCT1 = 10000 

TCT1 = RebaseToken(TYPE_COLLATERAL_TOKEN)
tctAddress = TCT1.mint(depositTCT1, APY)

x = T1.get_balance_deposits()
y = TCT1.get_balance_deposits()

# Initialize Liquidity Pool
liq = Liquidity(x,y)
L = liq.calc(); p = np.random.normal(5,0.1,1)[0]
cpt = ConstantProductTrade(L)
pCurve = PriceCurve(L)

print('[Initial] T1: {:.2f} TCT1: {:.2f} L: {:.5f}\n'.format(x,y,L))

[Initial] T1: 10000.00 TCT1: 10000.00 L: 10000.00000



* The deposit event that includes the amount of non-rebasing token will expand the supply of the rebasing token 1:1

In [3]:
token_delta = TokenDeltaModel(1,100).apply() 
time_delta = TimeDeltaModel().apply() 
T1.add_event(Deposit(0, token_delta, time_delta, t1Address))
TCT1.add_event(Deposit(APY, token_delta, time_delta, tctAddress)) 

TCT1_yield = TCT1.get_state_series(tctAddress).get_last_state().get_yield()

x = T1.get_balance_deposits()
y = TCT1.get_balance_deposits()

liq.delta_y(token_delta+TCT1_yield)
liq.delta_x(token_delta) 
L = liq.calc()

print('[Deposits] T1: {:.5f} TCT1: {:.5f} L: {:.5f}'.format(x,y,L))


[Deposits] T1: 10055.78302 TCT1: 10056.69574 L: 10056.23937


* The subsequent trade must be for the same amount being deposited.
* The amount of rebasing token purchase must be reflected as having left CPT liquidity.

In [4]:
T1.add_event(Withdraw(0, token_delta, 0, t1Address))
TCT1.add_event(Deposit(APY, token_delta, 0, tctAddress)) 

TCT1_yield = TCT1.get_state_series(tctAddress).get_last_state().get_yield()
liq.delta_y(TCT1_yield)
L = liq.swap(token_delta)

x = T1.get_balance_deposits()
y = TCT1.get_balance_deposits()

print('[Swap] T1: {:.5f} TCT1: {:.5f} L: {:.5f}'.format(x,y,L))


[Swap] T1: 10000.00000 TCT1: 10112.47876 L: 10056.23937


### Part B: running through each of the above instructions through a loop

In [5]:
# Initialize Rebasing/NonRebasing Tokens
APY = 0.1

TYPE_DEPOSIT_TOKEN = 'DEPOSIT'
TYPE_COLLATERAL_TOKEN = 'COLLATERAL'

time_delta = TimeDeltaModel().apply()
depositT1 = 10000  

T1 = NonRebaseToken(TYPE_DEPOSIT_TOKEN)
t1Address = T1.mint(depositT1)

time_delta = TimeDeltaModel().apply()
depositTCT1 = 10000 

TCT1 = RebaseToken(TYPE_COLLATERAL_TOKEN)
tctAddress = TCT1.mint(depositTCT1, APY)

x = T1.get_balance_deposits()
y = TCT1.get_balance_deposits()

# Initialize Liquidity Pool
liq = Liquidity(x,y)
L = liq.calc(); p = np.random.normal(5,0.1,1)[0]

print('[Initial] T1: {:.2f} TCT1: {:.2f} L: {:.5f}\n'.format(x,y,L))

[Initial] T1: 10000.00 TCT1: 10000.00 L: 10000.00000



[0. Deposits] T1: 10275.33332 TCT1: 10286.72821 L: 10281.02919
[0. Swap] T1: 10000.00000 TCT1: 10562.06153 L: 10281.02919

[50. Deposits] T1: 10001.99090 TCT1: 21740.91102 L: 16005.52330
[50. Swap] T1: 10000.00000 TCT1: 21742.90193 L: 16005.52330

[100. Deposits] T1: 10002.14594 TCT1: 30079.31080 L: 20470.37711
[100. Swap] T1: 10000.00000 TCT1: 30081.45674 L: 20470.37711

[150. Deposits] T1: 10004.21265 TCT1: 41816.54699 L: 26914.65735
[150. Swap] T1: 10000.00000 TCT1: 41820.75964 L: 26914.65735

[200. Deposits] T1: 10026.65026 TCT1: 50789.32081 L: 31953.88364
[200. Swap] T1: 10000.00000 TCT1: 50815.97107 L: 31953.88364



In [6]:
# Plot and execute N trades
percent = 0.05
N = 250

cpt = ConstantProductTrade(L)
pCurve = PriceCurve(L)
plotCurve = PlotPriceCurve(L, cpt, pCurve)
plotCurve.apply(p, None)

for k in range(N):
    
    # The deposit event that includes the amount of non-rebasing token will 
    # expand the supply of the rebasing token 1:1
    
    token_delta = TokenDeltaModel(1,100).apply() 
    time_delta = TimeDeltaModel().apply() 
    T1.add_event(Deposit(0, token_delta, time_delta, t1Address))
    TCT1.add_event(Deposit(APY, token_delta, time_delta, tctAddress)) 

    TCT1_yield = TCT1.get_state_series(tctAddress).get_last_state().get_yield()

    x = T1.get_balance_deposits()
    y = TCT1.get_balance_deposits()

    liq.delta_x(token_delta)
    L = liq.calc()
    liq.delta_y(token_delta+TCT1_yield)     
    L = liq.calc(); #p = np.random.normal(5,0.1,1)[0]
    p = liq.get_swap_price()

    if (k % int(N/(100*percent)) == 0): 
        print('[{}. Deposits] T1: {:.5f} TCT1: {:.5f} L: {:.5f}'.format(k,x,y,L))
    
    # The subsequent trade must be for the same amount being deposited
    
    T1.add_event(Withdraw(0, token_delta, 0, t1Address))
    TCT1.add_event(Deposit(APY, token_delta, 0, tctAddress)) 

    # The amount of rebasing token purchase must be reflected as having 
    # left CPT liquidity.

    TCT1_yield = TCT1.get_state_series(tctAddress).get_last_state().get_yield()
    liq.delta_y(TCT1_yield)
    L = liq.swap(token_delta)

    x = T1.get_balance_deposits()
    y = TCT1.get_balance_deposits()

    if (k % int(N/(100*percent)) == 0):
        plotCurve.apply_next(p, L, '({}) L = {}, p = {:.3f}'.format(k, int(L),p))
        plotCurve.plot_trade(p, token_delta, 'r--')        
        print('[{}. Swap] T1: {:.5f} TCT1: {:.5f} L: {:.5f}\n'.format(k,x,y,L))    



In [7]:
# Analyze Token balances

T1_states = T1.get_state_series(t1Address)
principle = T1_states.get_principle() 
balance = T1_states.get_balance() 
tSamp = T1_states.get_tstamp()  

fig, (T1_ax, TCT1_ax) = plt.subplots(nrows=2, sharex=True, sharey=False, figsize=(12, 10))

T1_ax.plot(tSamp[1:], balance[1:], label='balance') 
T1_ax.plot(tSamp[1:], principle[1:], label='principle') 
T1_ax.set_title("T1 Balance (non-rebasing token)", fontsize=20)
T1_ax.set_ylabel("Token balance", fontsize=16)
T1_ax.set_xlabel('Time', fontsize=16)
T1_ax.legend()

TCT1_states = TCT1.get_state_series(tctAddress)
principle = TCT1_states.get_principle() 
balance = TCT1_states.get_balance() 
tSamp = TCT1_states.get_tstamp() 

TCT1_ax.plot(tSamp[1:], balance[1:], label='balance') 
TCT1_ax.plot(tSamp[1:], principle[1:], label='principle') 
TCT1_ax.set_title("TCT1 Balance (rebasing token)", fontsize=20)
TCT1_ax.set_ylabel("Token balance", fontsize=16)
TCT1_ax.set_xlabel('Time', fontsize=16)
TCT1_ax.legend()
